# Under the Hood of Encrypted Neural Networks

This tutorial is optional, and can be skipped without loss of continuity.

In this tutorial, we'll take a look at how CrypTen performs inference with an encrypted neural network on encrypted data. We'll see how the data remains encrypted through all the operations, and yet is able to obtain accurate results after the computation. 

In [1]:
import crypten
import torch

crypten.init() 

# Ignore warnings
import warnings; 
warnings.filterwarnings("ignore")

# Keep track of all created temporary files so that we can clean up at the end
temp_files = []

## A Simple Linear Layer
We'll start by examining how a single Linear layer works in CrypTen. We'll instantiate a torch Linear layer, convert to CrypTen layer, encrypt it, and step through some toy data with it. As in earlier tutorials, we'll assume Alice has the rank 0 process and Bob has the rank 1 process. We'll also assume Alice has the layer and Bob has the data.

In [2]:
# Define ALICE and BOB src values
ALICE = 0
BOB = 1

In [3]:
import torch.nn as nn

# Instantiate single Linear layer
layer_linear = nn.Linear(4, 2)

# The weights and the bias are initialized to small random values
print("Plaintext Weights:", layer_linear._parameters['weight'])
print("Plaintext Bias:", layer_linear._parameters['bias'])

# Save the plaintext layer
layer_linear_file = "/tmp/tutorial5_layer_alice1.pth"
crypten.save(layer_linear, layer_linear_file)
temp_files.append(layer_linear_file) 

# Generate some toy data
features = 4
examples = 3
toy_data = torch.rand(examples, features)

# Save the plaintext toy data
toy_data_file = "/tmp/tutorial5_data_bob1.pth"
crypten.save(toy_data, toy_data_file)
temp_files.append(toy_data_file)

Plaintext Weights: Parameter containing:
tensor([[ 0.3941, -0.4677, -0.4373, -0.4780],
        [-0.1308,  0.2255, -0.2533, -0.0447]], requires_grad=True)
Plaintext Bias: Parameter containing:
tensor([-0.0189, -0.0790], requires_grad=True)


In [4]:
import crypten.mpc as mpc
import crypten.communicator as comm

@mpc.run_multiprocess(world_size=2)
def forward_single_encrypted_layer():
    rank = comm.get().get_rank()
    
    # Load and encrypt the layer
    layer = crypten.load(layer_linear_file, dummy_model=nn.Linear(4, 2), src=ALICE)
    layer_enc = crypten.nn.from_pytorch(layer, dummy_input=torch.empty((1,4)))
    layer_enc.encrypt(src=ALICE)
    
    # Load and encrypt data
    data = crypten.load(toy_data_file, src=BOB)    
    data_enc = crypten.cryptensor(data, src=BOB) 
    
    # Examine the shares inside the encrypted data
    print("Rank: {} Encrypted data:\n{}\n".format(rank, data_enc.share))

    # Examine the weights and the bias of the linear layer again
    # First, we'll see that weights and bias have become encrypted tensors
    print("Rank: {} Weights Encrypted: {}".format(rank, crypten.is_encrypted_tensor(layer_enc._parameters['weight'])))
    print("Rank: {} Bias Encrypted: {}".format(rank, crypten.is_encrypted_tensor(layer_enc._parameters['bias'])))
    # Now let's look at the tensor values
    print("Rank: {} Encrypted Weights:\n{}".format(rank, layer_enc._parameters['weight'].share))
    print("Rank: {} Encrypted Bias: {}".format(rank, layer_enc._parameters['bias'].share))
    print()

    # Apply the encrypted layer: encrypted linear transformation 
    result_enc = layer_enc.forward(data_enc)
    # Examine the resulting shares of both parties
    print("Rank: {} Encrypted result:\n{}\n".format(rank, result_enc.share))
    
    # Decrypt the result:
    result_plaintext = result_enc.get_plain_text()
    # Since both parties have the same decrypted values, print only rank 0 values for readability
    if rank == 0:
        print("Decrypted result:\n", result_plaintext)
        
forward_single_encrypted_layer()

Rank: 1 Encrypted data:
tensor([[ 3678700102179911956, -2909050226621709958,  2811555063257232306,
         -4928920039109496584],
        [-5461804489811683468,  2507827404725734825, -1277464813380873939,
          4588017341358575108],
        [ 5802222930185798095, -3989271002924527689,  8211929695094444304,
          3369799817375354834]])
Rank: 0 Encrypted data:
tensor([[-3678700102179867573,  2909050226621752754, -2811555063257171909,
          4928920039109526119],
        [ 5461804489811700428, -2507827404725680380,  1277464813380918463,
         -4588017341358574103],
        [-5802222930185753362,  3989271002924574164, -8211929695094428483,
         -3369799817375320623]])


Rank: 0 Weights Encrypted: True
Rank: 1 Weights Encrypted: True
Rank: 0 Bias Encrypted: True
Rank: 1 Bias Encrypted: True
Rank: 0 Encrypted Weights:
tensor([[-7136390616833108556,  7942355782122678145,  8415727642310264347,
         -1381325403809972801],
        [ 4877797614814754119,  168743864958422005

[None, None]

We can see that the application of the encrypted linear layer on the encrypted data produces an encrypted result, which we can then decrypt to get the values in plaintext.

Let's look at a second linear transformation, to give a flavor of how accuracy is preserved even when the data and the layer are encrypted. We'll look at a uniform scaling transformation, in which all tensor elements are multiplied by the same scalar factor. Again, we'll assume Alice has the layer and the rank 0 process, and Bob has the data and the rank 1 process.

In [5]:
# Initialize a linear layer with random weights
layer_scale = nn.Linear(3, 3)

# Construct a uniform scaling matrix: we'll scale by factor 5
factor = 5
layer_scale._parameters['weight'] = torch.eye(3)*factor
layer_scale._parameters['bias'] = torch.zeros_like(layer_scale._parameters['bias'])

# Save the plaintext layer
layer_scale_file = "/tmp/tutorial5_layer_alice2.pth"
crypten.save(layer_scale, layer_scale_file)
temp_files.append(layer_scale_file)

# Construct some toy data
features = 3
examples = 2
toy_data = torch.ones(examples, features)

# Save the plaintext toy data
toy_data_file = "/tmp/tutorial5_data_bob2.pth"
crypten.save(toy_data, toy_data_file)
temp_files.append(toy_data_file)

In [6]:
@mpc.run_multiprocess(world_size=2)
def forward_scaling_layer():
    rank = comm.get().get_rank()
    
    # Load and encrypt the layer
    layer = crypten.load(layer_scale_file, dummy_model=nn.Linear(3, 3), src=ALICE)
    layer_enc = crypten.nn.from_pytorch(layer, dummy_input=torch.empty((1,3)))
    layer_enc.encrypt(src=ALICE)
    
    # Load and encrypt data
    data = crypten.load(toy_data_file, src=BOB)    
    data_enc = crypten.cryptensor(data, src=BOB)     
    
    # Examine the encrypted data
    print("Rank: {} Encrypted data:\n{}\n".format(rank, data_enc.share))
        
    # Examine the encrypted layer
    print("Rank: {} Encrypted Weights:\n{}".format(rank, layer_enc._parameters['weight'].share))
    print("Rank: {} Encrypted Bias:\n{}".format(rank, layer_enc._parameters['bias'].share))

    # Apply the encrypted scaling transformation
    result_enc = layer_enc.forward(data_enc)
    
    # Examine the encrypted results
    print("Rank: {} Encrypted result:\n {}\n".format(rank, result_enc.share))

    # Decrypt the result:
    result_plaintext = result_enc.get_plain_text()
    # Since both parties have the same decrypted values, print only rank 0 for readability
    if rank == 0:
        print("Plaintext result:\n{}".format(result_plaintext))
        
z = forward_scaling_layer()

Rank: 1 Encrypted data:
tensor([[ 2380050498413143599,  1847879776682561747, -8964731574872462529],
        [-3234938927652527609, -4572902848872977324,  2461944650304271428]])
Rank: 0 Encrypted data:
tensor([[-2380050498413078063, -1847879776682496211,  8964731574872528065],
        [ 3234938927652593145,  4572902848873042860, -2461944650304205892]])


Rank: 0 Encrypted Weights:
tensor([[-8878088872997265406, -6954459607753448167,  -671476567678641743],
        [ 6019696710699457951,  8827473402821762442,  5194317374079125986],
        [-7659511891485929821, -2295374719619823579, -2090928637492781555]])Rank: 1 Encrypted Weights:
tensor([[ 8878088872997593086,  6954459607753448167,   671476567678641743],
        [-6019696710699457951, -8827473402821434762, -5194317374079125986],
        [ 7659511891485929821,  2295374719619823579,  2090928637493109235]])

Rank: 0 Encrypted Bias:
tensor([-1707544518694688470, -4483419037710543608, -7445515754401636293])Rank: 1 Encrypted Bias:
tensor([17

The resulting plaintext tensor is correctly scaled, even though we applied the encrypted transformation on the encrypted input! 

## Multi-layer Neural Networks
Let's now look at how the encrypted input moves through an encrypted multi-layer neural network. 

For ease of explanation, we'll first step through a network with only two linear layers and ReLU activations. Again, we'll assume Alice has a network and Bob has some data, and they wish to run encrypted inference. 

To simulate this, we'll once again generate some toy data and train Alice's network on it. Then we'll encrypt Alice's network, Bob's data, and step through every layer in the network with the encrypted data. Through this, we'll see how the computations get applied although the network and the data are encrypted.

### Setup
As in Tutorial 3, we will first generate 1000 ground truth samples using 50 features and a randomly generated hyperplane to separate positive and negative examples. We will then modify the labels so that they are all non-negative. Finally, we will split the data so that the first 900 samples belong to Alice and the last 100 samples belong to Bob.

In [7]:
# Setup
features = 50
examples = 1000

# Set random seed for reproducibility
torch.manual_seed(1)

# Generate toy data and separating hyperplane
data = torch.randn(examples, features)
w_true = torch.randn(1, features)
b_true = torch.randn(1)
labels = w_true.matmul(data.t()).add(b_true).sign()

# Change labels to non-negative values
labels_nn = torch.where(labels==-1, torch.zeros(labels.size()), labels)
labels_nn = labels_nn.squeeze().long()

# Split data into Alice's and Bob's portions:
data_alice, labels_alice = data[:900], labels_nn[:900]
data_bob, labels_bob = data[900:], labels_nn[900:]

In [8]:
# Define Alice's network
import torch.nn as nn
import torch.nn.functional as F

class AliceNet(nn.Module):
    def __init__(self):
        super(AliceNet, self).__init__()
        self.fc1 = nn.Linear(50, 20)
        self.fc2 = nn.Linear(20, 2)
        
    def forward(self, x):
        out = self.fc1(x)
        out = F.relu(out)
        out = self.fc2(out)
        return out

In [9]:
# Train and save Alice's network
model = AliceNet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

for i in range(500):  
    #forward pass: compute prediction
    output = model(data_alice)
    
    #compute and print loss
    loss = criterion(output, labels_alice)
    if i % 100 == 99:
        print("Epoch", i, "Loss:", loss.item())
    
    #zero gradients for learnable parameters
    optimizer.zero_grad()
    
    #backward pass: compute gradient with respect to model parameters
    loss.backward()
    
    #update model parameters
    optimizer.step()

sample_trained_model_file = '/tmp/tutorial5_alice_model.pth'
torch.save(model, sample_trained_model_file)
temp_files.append(sample_trained_model_file)

Epoch 99 Loss: 0.2470429241657257
Epoch 199 Loss: 0.08965438604354858
Epoch 299 Loss: 0.05166155472397804
Epoch 399 Loss: 0.03510778397321701
Epoch 499 Loss: 0.026072446256875992


### Stepping through a Multi-layer Network

Let's now look at what happens when we load the network Alice's has trained and encrypt it. First, we'll look at how the network structure changes when we convert it from a PyTorch network to CrypTen network.

In [10]:
# Load the trained network to Alice
model_plaintext = crypten.load(sample_trained_model_file, dummy_model=AliceNet(), src=ALICE)

# Convert the trained network to CrypTen network 
private_model = crypten.nn.from_pytorch(model_plaintext, dummy_input=torch.empty((1, 50)))
# Encrypt the network
private_model.encrypt(src=ALICE)

# Examine the structure of the encrypted CrypTen network
for name, curr_module in private_model._modules.items():
    print("Name:", name, "\tModule:", curr_module)

Name: 5 	Module: <crypten.nn.module.Linear object at 0x7fde8808f590>
Name: 6 	Module: <crypten.nn.module.ReLU object at 0x7fdee8479650>
Name: output 	Module: <crypten.nn.module.Linear object at 0x7fdee84798d0>


We see that the encrypted network has 3 modules, named '5', '6' and 'output', denoting the first Linear layer, the ReLU activation, and the second Linear layer respectively. These modules are encrypted just as the layers in the previous section were. 

Now let's encrypt Bob's data, and step it through each encrypted module. For readability, we will use only 3 examples from Bob's data to illustrate the inference. Note how Bob's data remains encrypted after each individual layer's computation!

In [11]:
# Pre-processing: Select only the first three examples in Bob's data for readability
data = data_bob[:3]
sample_data_bob_file = '/tmp/tutorial5_data_bob3.pth'
torch.save(data, sample_data_bob_file)
temp_files.append(sample_data_bob_file)

In [12]:
@mpc.run_multiprocess(world_size=2)
def step_through_two_layers():    
    rank = comm.get().get_rank()

    # Load and encrypt the network
    model = crypten.load(sample_trained_model_file, dummy_model=AliceNet(), src=ALICE)
    private_model = crypten.nn.from_pytorch(model, dummy_input=torch.empty((1, 50)))
    private_model.encrypt(src=ALICE)

    # Load and encrypt the data
    data = crypten.load(sample_data_bob_file, src=BOB)
    data_enc = crypten.cryptensor(data, src=BOB)

    # Forward through the first layer
    out_enc = private_model._modules['5'].forward(data_enc)
    print("Rank: {} First Linear Layer: Output Encrypted: {}\n".format(rank, crypten.is_encrypted_tensor(out_enc)))
    print("Rank: {} Shares after First Linear Layer:{}\n".format(rank, out_enc.share))

    # Apply ReLU activation
    out_enc = private_model._modules['6'].forward(out_enc)
    print("Rank: {} ReLU:\n Output Encrypted: {}\n".format(rank, crypten.is_encrypted_tensor(out_enc)))
    print("Rank: {} Shares after ReLU: {}\n".format(rank, out_enc.share))

    # Forward through the second Linear layer
    out_enc = private_model._modules['output'].forward(out_enc)
    print("Rank: {} Second Linear layer:\n Output Encrypted: {}\n".format(rank, crypten.is_encrypted_tensor(out_enc))), 
    print("Rank: {} Shares after Second Linear layer:{}\n".format(rank, out_enc.share))

    # Decrypt the output
    out_dec = out_enc.get_plain_text()
    # Since both parties have same decrypted results, only print the rank 0 output
    if rank == 0:
        print("Decrypted output:\n Output Encrypted:", crypten.is_encrypted_tensor(out_dec))
        print("Tensors:\n", out_dec)
    
z = step_through_two_layers()

Rank: 0 First Linear Layer: Output Encrypted: True
Rank: 1 First Linear Layer: Output Encrypted: True


Rank: 1 Shares after First Linear Layer:tensor([[-2224392792640713959,  6083933912752127281, -2049841560836145201,
         -3804603540994371973, -1282845883776053085,  3271623645745577736,
         -2962090725151140639,  1405323579613973433,  7555130549438824985,
         -6931999527211605420, -8285657089197970503,  1314901661728837421,
         -5620760993017059357,  7475902553670408973, -8756112654425768089,
         -2233694146531187381, -4242376502263878867,  1417352514974486780,
          4437703783822774923,  4681976352489113695],
        [-2224550209238588380,  6083918444126157129, -2049776429354933648,
         -3804578160550366625, -1282927694982073609,  3271677400736939913,
         -2961963807683856724,  1405274710644603910,  7555073749451748894,
         -6931788212312501229, -8285572078619406303,  1314791283283139625,
         -5620638356681602645,  7475843603576212719,

Again, we emphasize that the output of each layer is an encrypted tensor. Only after the final call to `get_plain_text` do we get the plaintext tensor.

### From PyTorch to CrypTen: Structural Changes in Network Architecture 

We have used a simple two-layer network in the above example, but the same ideas apply to more complex networks and operations. However, in more complex networks, there may not always be a one-to-one mapping between the PyTorch layers and the CrypTen layers. As an example, we'll take a typical network used to classify digits in MNIST data, and look at what happens to its structure when encrypted. (As we only wish to illustrate the structural changes in layers, we will not train this network on data; we will just use it with its randomly initialized weights). 

In [13]:
# Define Alice's network
class AliceNet2(nn.Module):
    def __init__(self):
        super(AliceNet2, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5, padding=0)
        self.conv2 = nn.Conv2d(16, 16, kernel_size=5, padding=0)
        self.fc1 = nn.Linear(16 * 4 * 4, 100)
        self.fc2 = nn.Linear(100, 10)
        self.batchnorm1 = nn.BatchNorm2d(16)
        self.batchnorm2 = nn.BatchNorm2d(16)
        self.batchnorm3 = nn.BatchNorm1d(100)
 
    def forward(self, x):
        out = self.conv1(x)
        out = self.batchnorm1(out)
        out = F.relu(out)
        out = F.avg_pool2d(out, 2)
        out = self.conv2(out)
        out = self.batchnorm2(out)
        out = F.relu(out)
        out = F.avg_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.batchnorm3(out)
        out = F.relu(out)
        out = self.fc2(out)
        return out
    
model = AliceNet2()

# Let's encrypt the complex network. 
# Create dummy input of the correct input shape for the model
dummy_input = torch.empty((1, 1, 28, 28))

# Encrypt the network
private_model = crypten.nn.from_pytorch(model, dummy_input)
private_model.encrypt(src=ALICE)

# Examine the structure of the encrypted network
for name, curr_module in private_model._modules.items():
    print("Name:", name, "\tModule:", curr_module)

Name: 24 	Module: <crypten.nn.module.Conv2d object at 0x7fdec87d3990>
Name: 25 	Module: <crypten.nn.module._BatchNorm object at 0x7fdec87d3910>
Name: 26 	Module: <crypten.nn.module.ReLU object at 0x7fdec87d3cd0>
Name: 27 	Module: <crypten.nn.module._ConstantPad object at 0x7fdec87d3290>
Name: 28 	Module: <crypten.nn.module.AvgPool2d object at 0x7fdec87d3150>
Name: 29 	Module: <crypten.nn.module.Conv2d object at 0x7fded8535890>
Name: 30 	Module: <crypten.nn.module._BatchNorm object at 0x7fdec87d3350>
Name: 31 	Module: <crypten.nn.module.ReLU object at 0x7fdec87d32d0>
Name: 32 	Module: <crypten.nn.module._ConstantPad object at 0x7fdec87d3810>
Name: 33 	Module: <crypten.nn.module.AvgPool2d object at 0x7fdec87d3210>
Name: 34 	Module: <crypten.nn.module.Constant object at 0x7fdec87d3590>
Name: 35 	Module: <crypten.nn.module.Shape object at 0x7fdec87d3250>
Name: 36 	Module: <crypten.nn.module.Gather object at 0x7fdec87d3850>
Name: 37 	Module: <crypten.nn.module.Constant object at 0x7fdec87d3

Notice how the CrypTen network has split the PyTorch network into several additional layers. Each PyTorch operation may correspond to one or more operations in CrypTen. Nevertheless, the same ideas apply when forwarding the data through the CrypTen equivalent of each PyTorch operation.  

Before exiting this tutorial, please clean up the files generated using the following code.

In [14]:
import os
for fn in temp_files:
    if os.path.exists(fn): os.remove(fn)